In [8]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import numpy as np
import cv2
import glob
import time
from sklearn.svm import LinearSVC
from sklearn.preprocessing import StandardScaler
from skimage.feature import hog
from lesson_functions import *

# NOTE: the next import is only valid for scikit-learn version <= 0.17
# for scikit-learn >= 0.18 use:
from sklearn.model_selection import train_test_split
# from sklearn.cross_validation import train_test_split


# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
def extract_features(imgs, cspace='RGB', orient=9, 
                        pix_per_cell=8, cell_per_block=2, hog_channel=0):
    # Create a list to append feature vectors to
    features = []
    # Iterate through the list of images
    for file in imgs:
        # Read in each one by one
        image = mpimg.imread(file)
#         image = cv2.resize(image, (64, 64))
        # apply color conversion if other than 'RGB'
        if cspace != 'RGB':
            if cspace == 'HSV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
            elif cspace == 'LUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2LUV)
            elif cspace == 'HLS':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2HLS)
            elif cspace == 'YUV':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YUV)
            elif cspace == 'YCrCb':
                feature_image = cv2.cvtColor(image, cv2.COLOR_RGB2YCrCb)
        else: feature_image = np.copy(image) 
        # Call get_hog_features() with vis=False, feature_vec=True
        if hog_channel == 'ALL':
            hog_features = []
            for channel in range(feature_image.shape[2]):
                hog_features.append(get_hog_features(feature_image[:,:,channel], 
                                    orient, pix_per_cell, cell_per_block, 
                                    vis=False, feature_vec=True, tsqrt=True))
            hog_features = np.ravel(hog_features)        
        else:
            hog_features = get_hog_features(feature_image[:,:,hog_channel], orient, 
                        pix_per_cell, cell_per_block, vis=False, feature_vec=True, tsqrt=True)
        # Get color features
#         spatial_features = bin_spatial(image, size=(spatial, spatial))
#         hist_features = color_hist(image, nbins=histbin)
        
        if np.any(np.isnan(hog_features)):
            continue
        # Append the new feature vector to the features list
        feature = np.hstack([hog_features])
        features.append(feature)
    # Return list of feature vectors
    return features


# Divide up into cars and notcars
images = glob.glob('*vehicles/*/*.png')
print(len(images))
cars = []
notcars = []
for image in images:
    if 'non' in image or 'negative' in image:
        notcars.append(image)
    else: 
        cars.append(image)
cars = np.array(cars)        
notcars = np.array(notcars)

# Reduce the sample size because HOG features are slow to compute
# The quiz evaluator times out after 13s of CPU time
# sample_size = 1000
# samples = np.random.randint(0, len(cars), sample_size)
# samples_nc = np.random.randint(0, len(notcars), sample_size)
cars = cars[:]
notcars = notcars[:]

### TODO: Tweak these parameters and see how the results change.
colorspace = 'YCrCb' # Can be RGB, HSV, LUV, HLS, YUV, YCrCb
orient = 9
pix_per_cell = 12
cell_per_block = 2
hog_channel = "ALL" # Can be 0, 1, 2, or "ALL"

t=time.time()
car_features = extract_features(cars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
notcar_features = extract_features(notcars, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to extract HOG features...')

17760
99.63 Seconds to extract HOG features...


In [9]:
print("Feature vector length:")
print(car_features[0].shape)

imgs = ['corner-cases/corner-case-car.png']
# Define a function to extract features from a list of images
# Have this function call bin_spatial() and color_hist()
features = extract_features(imgs, cspace=colorspace, orient=orient, 
                        pix_per_cell=pix_per_cell, cell_per_block=cell_per_block, 
                        hog_channel=hog_channel)[0]
print(features.shape)

Feature vector length:
(1728,)
(1728,)


In [10]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
print(np.array(car_features).shape)
print(np.array(notcar_features).shape)
# Create an array stack of feature vectors
X = np.vstack((car_features, notcar_features)).astype(np.float64)

# Define the labels vector
y = np.hstack((np.ones(len(car_features)), np.zeros(len(notcar_features))))

# Split up data into randomized training and test sets
rand_state = np.random.randint(0, 100)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=rand_state)
    

(8792, 1728)
(8968, 1728)


In [22]:
# Fit a per-column scaler
X_scaler = StandardScaler().fit(X_train)
# Apply the scaler to X
X_train = X_scaler.transform(X_train)
X_test = X_scaler.transform(X_test)

print('Using:',orient,'orientations',pix_per_cell,
    'pixels per cell and', cell_per_block,'cells per block')
print('Feature vector length:', len(X_train[0]))

# Use a nonlinear SVC 
parameters = {'kernel':('linear', 'rbf'), 'C':[0.01, 1.0, 10.]} 
clf = SVC(kernel='rbf', C=1., gamma=0.01)
# clf = GridSearchCV(svc, parameters) 

# Check the training time for the SVC
t=time.time()
clf.fit(X_train, y_train)
t2 = time.time()
print(round(t2-t, 2), 'Seconds to train SVC...')

Using: 9 orientations 12 pixels per cell and 2 cells per block
Feature vector length: 1728
555.77 Seconds to train SVC...


In [28]:
import pickle
# load a pe-trained svc model from a serialized (pickle) file
dist_pickle = pickle.load( open("svc_pickle_Y.p", "rb" ) )

# get attributes of our svc object
clf = dist_pickle["svc"]
X_scaler = dist_pickle["scaler"]
orient = dist_pickle["orient"]
pix_per_cell = dist_pickle["pix_per_cell"]
cell_per_block = dist_pickle["cell_per_block"]
cspace = dist_pickle["colorspace"]
hog_channel = dist_pickle["hog_channel"]


In [12]:
print(len(X_test))

3552


In [19]:
# Check the score of the SVC
print("Range of values: ", np.min(X_train), np.max(X_train), "Mean / STD: ", np.mean(X_train), np.std(X_train))
print('Test Accuracy of SVC = ', round(clf.score(X_test, y_test), 4))
# Check the prediction time for a single sample
t=time.time()
n_predict = 10
print("Feature shape: ", X_test[0:1].shape)
print("Feature range: ", np.min(X_test), np.max(X_test), np.mean(X_test))
print('My SVC predicts: ', clf.decision_function(X_test[0:n_predict]))
print('For these',n_predict, 'labels: ', y_test[0:n_predict])
t2 = time.time()
print(round(t2-t, 5), 'Seconds to predict', n_predict,'labels with SVC')
print("SVC random prediction:")
minimum = 10
maximum = -10
for i in range(400):
    x_random = 3*np.random.random([1, len(X_train[0])])
    dec = clf.decision_function(x_random)
    if dec > maximum:
        maximum = dec
    elif dec < minimum:
        minimum = dec
print("Maximum: ", maximum, "Minimum", minimum)

Range of values:  -0.942713734534 112.732198392 Mean / STD:  3.96375921371e-18 1.0
Test Accuracy of SVC =  0.9614
Feature shape:  (1, 1728)
Feature range:  -0.942713734534 181.941575305 -0.0074497084943
My SVC predicts:  [-15.70578392  -1.41009079   7.18575472 -16.63849397   2.01412133
   8.11799247   1.05446536 -16.688456     5.18797444  -3.06600437]
For these 10 labels:  [ 0.  1.  1.  0.  1.  1.  1.  0.  1.  0.]
0.03515 Seconds to predict 10 labels with SVC
SVC random prediction:
Maximum:  -10 Minimum [-92.53089529]


In [21]:
import pickle
# set attributes of our svc object
dist_pickle = dict()
dist_pickle["svc"] = clf
dist_pickle["scaler"] = X_scaler
dist_pickle["orient"] = orient
dist_pickle["pix_per_cell"] = pix_per_cell
dist_pickle["cell_per_block"] = cell_per_block
# dist_pickle["spatial_size"] = (spatial, spatial)
# dist_pickle["hist_bins"] = histbin
dist_pickle["colorspace"] = colorspace
dist_pickle["hog_channel"] = hog_channel
pickle.dump(dist_pickle, open("svc_pickle_HLSsmall.p", 'wb'))